In [1]:
# Load required packages

import os
import gc
import numpy as np
import tensorly as tl
import torch
from functools import reduce
from TFacTomo import reconstruct

# Set tensorly backend as pytorch
tl.set_backend('pytorch')

# Load 1D gene expression data along different spatial axes
X_x = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/normalized_fitted_lml_data.npy"); X_x = torch.from_numpy(X_x).to(torch.float)
X_y = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/normalized_fitted_dv_data.npy"); X_y = torch.from_numpy(X_y).to(torch.float)
X_z = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/normalized_fitted_ap_data.npy"); X_z = torch.from_numpy(X_z).to(torch.float)

genes = np.loadtxt("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/genes.txt", dtype=str)
# Load knowledge graphs along gene and spatial axes
W_g = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/mus_musculus_ppi_adjacency_matrix.npy"); W_g = torch.from_numpy(W_g).to(torch.float)
W_x = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/W_x.npy"); W_x = torch.from_numpy(W_x).to(torch.float)
W_y = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/W_y.npy"); W_y = torch.from_numpy(W_y).to(torch.float)
W_z = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/W_z.npy"); W_z = torch.from_numpy(W_z).to(torch.float)


# Load the 3D binary mask 
M = np.load("/Users/kimjob/PhD/PhD/Scripts/Main Project/Tomo-Seq/TFacTomo_OM_data/data/mouse_olfactory_mucosa/mouse_olfactory_mucosa_mask.npy"); M = torch.from_numpy(M).to(torch.float)


ModuleNotFoundError: No module named 'numpy'

In [3]:
# check which Kernel is actively used

import sys, os
print(sys.executable)
!which pip


/Users/kimjob/opt/anaconda3/envs/tfactomo/bin/python
/Users/kimjob/opt/anaconda3/envs/tfactomo/bin/pip
